In [1]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import re
import requests
import seaborn as sns

from db import create_connection
from db import (
    articles,
    services,
    urls
)

In [2]:
pd.set_option('max_colwidth', 400)

In [3]:
connection = create_connection()

In [4]:
df = pd.read_csv('se_lublin.csv')
df.head(1)

,Unnamed: 0,publication_date,publication_month,publication_day,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words,koronawirus_anywhere,koronawirus_anywhere_count
0,0,2020-08-31 14:40:00,2020-08,2020-08-31,karolina januszek,Wielkie promocje dla rodzin w Lidlu,https://lublin.se.pl/wielkie-promocje-dla-rodzin-w-lidlu-ak-ACGH-uaXC-Bt3y.html,0,0,0,2,249,0,0


In [18]:
def url_to_transcript(url):
    page = requests.get(url).text
    bs = BeautifulSoup(page, "lxml")
    container = bs.find(class_="main-content")
    article = container.find("article")
#     article.find("p", {"class": "playerBoard__text playerBoard__text--icon"}).decompose()
#     article.find("p", {"class": "playerBoard__text playerBoard__title"}).decompose()
#     article.find("p", {"class": "playerBoard__text"}).decompose()
    texts = article.find_all("p")
    articles = []
    for t in texts:
        article = t.text
        articles.append(article)
    return articles

In [5]:
data = {}

In [6]:
names = ["2020-01", "2020-02", "2020-03", "2020-04", "2020-05", "2020-06", "2020-07", "2020-08"]

In [21]:
for name in names:
    name_urls = df[['url']].where(df['publication_month'] == name).dropna()
    text = []
    for key, url in name_urls.items():
        for k, v in url.items():
            text.append(url_to_transcript(v))
    text_string = ''.join([str(item) for item in text])
    data[name] = text_string


In [22]:
for i, name in enumerate(names):
    with open("transcripts/" + name + ".txt", "wb") as file:
        pickle.dump(data[name], file)

In [7]:
data = {}
for i, name in enumerate(names):
    with open("transcripts/" + name + ".txt", "rb") as file:
        data[name] = pickle.load(file)

In [8]:
data.keys()

dict_keys(['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08'])

In [9]:
next(iter(data.keys()))

'2020-01'

In [10]:
next(iter(data.values()))

'[\'Prawie trzy tysiące e-zwolnień wystawiają dziennie lekarze z Lubelszczyzny. Nowy kreator do ich wypełniania od Zakładu Ubezpieczeń Społecznych może usprawnić ten proces. Testy przebiegły pomyślnie. Co się zmieni?\', \'Zakład Ubezpieczeń Społecznych udostępnił medykom nowe narzędzie do wystawiania elektronicznych zwolnień, które pod koniec 2018 roku zastąpiły papierowe druki. Wszystkie ważne informacje i pola do wypełnienia znajdują się teraz na jednej stronie.\', \'– Formularz do wystawiania elektronicznych zwolnień lekarskich jest teraz dla lekarzy znacznie prostszy, bardziej przejrzysty i funkcjonalny. Dodatkowo lekarz ma wygodny podgląd do listy wcześniej wystawionych zwolnień, jakie pacjent otrzymał w ostatnich miesiącach – tłumaczy Małgorzata Korba, regionalny rzecznik ZUS na Lubelszczyźnie.\', \'Testy przyniosły pozytywne opinie. Lekarze z regionu znajdą nowy kreator na swojej Platformie Usług Elektronicznych.\', \'Na PUE ZUS pojawiły się również inne nowe funkcje dla lekarzy

In [27]:
#name_urls = df[['url']].where(df['publication_month'] == name).dropna()

In [11]:
def combine_text(list_of_text):
    combined_text = ''.join(list_of_text)
    return combined_text

In [12]:
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [13]:
# next(iter(data_combined.values()))

In [14]:
data_combined['2020-06']

['[\'Tomasz Pakuła z Zamościa (woj. lubelskie) zobaczył już prawie pół świata i odwiedzając kolejne ciekawe muzea, coraz bardziej marzył... o własnym! Własne muzeum, według własnego pomysłu i przygotowane własnymi rękami. A dlaczego PRL? – Wizja kolejek, kartek na towary, niedostępności produktów, wielu absurdów, może być dla obecnego pokolenia abstrakcją, którą chciałbym pokazać – tłumaczy twórca Roztoczańskiego Muzeum PRL.\\r\\n\', \'Przygotowania zaczął jakieś trzy lata temu. Zbiory, które udostępni, pochodzą przede wszystkim z Roztocza i Zamojszczyzny, jednak wiele eksponatów podarowali mu ludzie z całej Polski. Sam budynek, gdzie mieści się muzeum, mógłby opowiedzieć niejedną ciekawą historię.\', \'– Moi teściowie udostępnili mi budynek, w którym był m.in. sklep spożywczo-przemysłowy i lumpeks. Historia tego miejsca – rodem z PRL – będzie mogła toczyć się dalej – cieszy się Pakuła.\', \'Zobacz też: Zamość: 28-latek chciał zrobić dobre zdjęcie. CUDEM NIE ZGINĄŁ! [ZDJĘCIA]\', \'Do s

In [15]:
pd.set_option('max_colwidth', 150)
data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

,transcript
2020-01,['Prawie trzy tysiące e-zwolnień wystawiają dziennie lekarze z Lubelszczyzny. Nowy kreator do ich wypełniania od Zakładu Ubezpieczeń Społecznych m...
2020-02,"['Jej matka spotykała się z 22-latkiem, a on… wykorzystywał seksualnie jej córkę! Horror na Lubelszczyźnie! Śledczy podejrzewają, że cudzoziemiec ..."
2020-03,['Kolejny dramatyczny wzrost liczby chorych – w Polsce i na Lubelszczyźnie. Epidemia koronawirusa przybiera na sile. Dziesięć nowych przypadków za...
2020-04,"['Chwile grozy przeżyła 43-letnia kobieta, która wybrała się na zakupy do Włodawy (woj. lubelskie). Przeraziła się, bo ktoś ukradł jej opla zapark..."
2020-05,"['Miał zatroszczyć się o wszystko: od transportu do wspaniałego hotelu, przez skipassy, posiłki i piękne widoki na stokach. Marek P. (47 l.) z Lub..."
2020-06,"['Tomasz Pakuła z Zamościa (woj. lubelskie) zobaczył już prawie pół świata i odwiedzając kolejne ciekawe muzea, coraz bardziej marzył... o własnym..."
2020-07,"['Przekonywał, że z zamiłowania jest ogrodnikiem, a ostra jak brzytwa maczeta służy mu do... przycinania żywopłotu. W rzeczywistości Piotr M. (42 ..."
2020-08,['Lidl już czwarty rok z rzędu honoruje Kartę Dużej Rodziny w swoich sklepach. Dzięki temu jej posiadacze otrzymują rabaty na wybrany asortyment s...


In [16]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('"*s"', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [17]:
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

,transcript
2020-01,dzisiaj również budynek ratusza i arena lublin zostaną podświetlone barwami gdańska żółtoczerwonymi tłumaczy katarzyna duma rzecznik prasowy pre...
2020-02,polecany artykuł rozwijamy nasz serwis dzięki wyświetlaniu reklam blokując reklamy nie pozwalasz nam tworzyć wartościowych treści wyłącz adblock...
2020-03,zobacz też czy lato wypłoszy koronawirusa jak się nie dać wirusowi zalecenia immunologów rozwijamy nasz serwis dzięki wyświetlaniu reklam blokują...
2020-04,bezobjawowy koronawirus co może niepokoić rozwijamy nasz serwis dzięki wyświetlaniu reklam blokując reklamy nie pozwalasz nam tworzyć wartościow...
2020-05,kasa firmy świeciła pustkami postanowił więc zagrać va banque zaczął więc organizować super wyjazd wyłącznie na papierze i w opowieściach którymi...
2020-06,do sprawy podchodzi niezwykle poważnie wziął nawet udział w szkoleniu dla założycieli i organizatorów muzeów prywatnych organizowanym przez narod...
2020-07,specjalnie z tej okazji w sklepach lidla znajdziemy przygotowane gadżety między innymi wyjątkowa koszulkę zaprojektowaną przez samego jurka owsia...
2020-08,powrót do szkoły z rossmannem w tym roku w ofercie rossmanna znajdziemy szeroki wybór artykułów szkolnych w atrakcyjnych cenach bo wyboru są międ...


In [18]:
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\s-\s', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [19]:
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean

,transcript
2020-01,dzisiaj również budynek ratusza i arena lublin zostaną podświetlone barwami gdańska żółtoczerwonymi tłumaczy katarzyna duma rzecznik prasowy pre...
2020-02,polecany artykuł rozwijamy nasz serwis dzięki wyświetlaniu reklam blokując reklamy nie pozwalasz nam tworzyć wartościowych treści wyłącz adblock...
2020-03,zobacz też czy lato wypłoszy koronawirusa jak się nie dać wirusowi zalecenia immunologów rozwijamy nasz serwis dzięki wyświetlaniu reklam blokują...
2020-04,bezobjawowy koronawirus co może niepokoić rozwijamy nasz serwis dzięki wyświetlaniu reklam blokując reklamy nie pozwalasz nam tworzyć wartościow...
2020-05,kasa firmy świeciła pustkami postanowił więc zagrać va banque zaczął więc organizować super wyjazd wyłącznie na papierze i w opowieściach którymi...
2020-06,do sprawy podchodzi niezwykle poważnie wziął nawet udział w szkoleniu dla założycieli i organizatorów muzeów prywatnych organizowanym przez narod...
2020-07,specjalnie z tej okazji w sklepach lidla znajdziemy przygotowane gadżety między innymi wyjątkowa koszulkę zaprojektowaną przez samego jurka owsia...
2020-08,powrót do szkoły z rossmannem w tym roku w ofercie rossmanna znajdziemy szeroki wybór artykułów szkolnych w atrakcyjnych cenach bo wyboru są międ...


In [21]:
next(iter(data.keys()))

'2020-01'

In [22]:
names = list(data.keys())

In [23]:
data_df['publication_month'] = names

In [24]:
data_df

,transcript,publication_month
2020-01,['Prawie trzy tysiące e-zwolnień wystawiają dziennie lekarze z Lubelszczyzny. Nowy kreator do ich wypełniania od Zakładu Ubezpieczeń Społecznych m...,2020-01
2020-02,"['Jej matka spotykała się z 22-latkiem, a on… wykorzystywał seksualnie jej córkę! Horror na Lubelszczyźnie! Śledczy podejrzewają, że cudzoziemiec ...",2020-02
2020-03,['Kolejny dramatyczny wzrost liczby chorych – w Polsce i na Lubelszczyźnie. Epidemia koronawirusa przybiera na sile. Dziesięć nowych przypadków za...,2020-03
2020-04,"['Chwile grozy przeżyła 43-letnia kobieta, która wybrała się na zakupy do Włodawy (woj. lubelskie). Przeraziła się, bo ktoś ukradł jej opla zapark...",2020-04
2020-05,"['Miał zatroszczyć się o wszystko: od transportu do wspaniałego hotelu, przez skipassy, posiłki i piękne widoki na stokach. Marek P. (47 l.) z Lub...",2020-05
2020-06,"['Tomasz Pakuła z Zamościa (woj. lubelskie) zobaczył już prawie pół świata i odwiedzając kolejne ciekawe muzea, coraz bardziej marzył... o własnym...",2020-06
2020-07,"['Przekonywał, że z zamiłowania jest ogrodnikiem, a ostra jak brzytwa maczeta służy mu do... przycinania żywopłotu. W rzeczywistości Piotr M. (42 ...",2020-07
2020-08,['Lidl już czwarty rok z rzędu honoruje Kartę Dużej Rodziny w swoich sklepach. Dzięki temu jej posiadacze otrzymują rabaty na wybrany asortyment s...,2020-08


In [25]:
data_df.to_pickle("corpus.pkl")

In [ ]:
stop_words = frozenset(sorted(["a", "ab", "abp", "aby", "adblock","ale", "artykuł" "aż",
                        "bo", "by",
                        "co", "czy",
                        "do", "dla",
                        "gdy", "gdyż", "gdzie",
                        "i", "ich", "im",
                        "jak", "jakże", "jest", "jeśli", "jeżeli",
                        "kto", "ktoś", "która", "które","który",
                        "lub",
                        "ma", "mam", "może",
                        "na", "nad",
                        "o", "od", "oraz",
                        "pod", "polecany artykuł", "przez", "przy",
                        "są", "się",
                        "to", "tym",
                        "u",
                        "w",
                        "z", "że", "żeby"]))

In [26]:
from stop_words import get_stop_words

In [27]:
stop_words = get_stop_words('polish')
stop_words

['ach',
 'aj',
 'albo',
 'bardzo',
 'bez',
 'bo',
 'być',
 'ci',
 'cię',
 'ciebie',
 'co',
 'czy',
 'daleko',
 'dla',
 'dlaczego',
 'dlatego',
 'do',
 'dobrze',
 'dokąd',
 'dość',
 'dużo',
 'dwa',
 'dwaj',
 'dwie',
 'dwoje',
 'dziś',
 'dzisiaj',
 'gdyby',
 'gdzie',
 'go',
 'ich',
 'ile',
 'im',
 'inny',
 'ja',
 'ją',
 'jak',
 'jakby',
 'jaki',
 'je',
 'jeden',
 'jedna',
 'jedno',
 'jego',
 'jej',
 'jemu',
 'jeśli',
 'jest',
 'jestem',
 'jeżeli',
 'już',
 'każdy',
 'kiedy',
 'kierunku',
 'kto',
 'ku',
 'lub',
 'ma',
 'mają',
 'mam',
 'mi',
 'mną',
 'mnie',
 'moi',
 'mój',
 'moja',
 'moje',
 'może',
 'mu',
 'my',
 'na',
 'nam',
 'nami',
 'nas',
 'nasi',
 'nasz',
 'nasza',
 'nasze',
 'natychmiast',
 'nią',
 'nic',
 'nich',
 'nie',
 'niego',
 'niej',
 'niemu',
 'nigdy',
 'nim',
 'nimi',
 'niż',
 'obok',
 'od',
 'około',
 'on',
 'ona',
 'one',
 'oni',
 'ono',
 'owszem',
 'po',
 'pod',
 'ponieważ',
 'przed',
 'przedtem',
 'są',
 'sam',
 'sama',
 'się',
 'skąd',
 'tak',
 'taki',
 'tam',
 'ten

In [ ]:
# aa = 'a aby ach acz aczkolwiek aj albo ale alez ależ ani az aż bardziej bardzo beda bedzie bez beda
# będą bede będę będzie bo bowiem by byc być byl byla byli bylo byly był była było były bynajmniej cala
# cali caly cała cały ci cie ciebie
# cię
# co
# cokolwiek
# cos
# coś
# czasami
# czasem
# czemu
# czy
# czyli
# daleko
# dla
# dlaczego
# dlatego
# do
# dobrze
# dokad
# dokąd
# dosc
# dość
# duzo
# dużo
# dwa
# dwaj
# dwie
# dwoje
# dzis
# dzisiaj
# dziś
# gdy
# gdyby
# gdyz
# gdyż
# gdzie
# gdziekolwiek
# gdzies
# gdzieś
# go
# i
# ich
# ile
# im
# inna
# inne
# inny
# innych
# iz
# iż
# ja
# jak
# jakas
# jakaś
# jakby
# jaki
# jakichs
# jakichś
# jakie
# jakis
# jakiś
# jakiz
# jakiż
# jakkolwiek
# jako
# jakos
# jakoś
# ją
# je
# jeden
# jedna
# jednak
# jednakze
# jednakże
# jedno
# jego
# jej
# jemu
# jesli
# jest
# jestem
# jeszcze
# jeśli
# jezeli
# jeżeli
# juz
# już
# kazdy
# każdy
# kiedy
# kilka
# kims
# kimś
# kto
# ktokolwiek
# ktora
# ktore
# ktorego
# ktorej
# ktory
# ktorych
# ktorym
# ktorzy
# ktos
# ktoś
# która
# które
# którego
# której
# który
# których
# którym
# którzy
# ku
# lat
# lecz
# lub
# ma
# mają
# mało
# mam
# mi
# miedzy
# między
# mimo
# mna
# mną
# mnie
# moga
# mogą
# moi
# moim
# moj
# moja
# moje
# moze
# mozliwe
# mozna
# może
# możliwe
# można
# mój
# mu
# musi
# my
# na
# nad
# nam
# nami
# nas
# nasi
# nasz
# nasza
# nasze
# naszego
# naszych
# natomiast
# natychmiast
# nawet
# nia
# nią
# nic
# nich
# nie
# niech
# niego
# niej
# niemu
# nigdy
# nim
# nimi
# niz
# niż
# no
# o
# obok
# od
# około
# on
# ona
# one
# oni
# ono
# oraz
# oto
# owszem
# pan
# pana
# pani
# po
# pod
# podczas
# pomimo
# ponad
# poniewaz
# ponieważ
# powinien
# powinna
# powinni
# powinno
# poza
# prawie
# przeciez
# przecież
# przed
# przede
# przedtem
# przez
# przy
# roku
# rowniez
# również
# sam
# sama
# są
# sie
# się
# skad
# skąd
# soba
# sobą
# sobie
# sposob
# sposób
# swoje
# ta
# tak
# taka
# taki
# takie
# takze
# także
# tam
# te
# tego
# tej
# ten
# teraz
# też
# to
# toba
# tobą
# tobie
# totez
# toteż
# totobą
# trzeba
# tu
# tutaj
# twoi
# twoim
# twoj
# twoja
# twoje
# twój
# twym
# ty
# tych
# tylko
# tym
# u
# w
# wam
# wami
# was
# wasz
# wasza
# wasze
# we
# według
# wiele
# wielu
# więc
# więcej
# wlasnie
# właśnie
# wszyscy
# wszystkich
# wszystkie
# wszystkim
# wszystko
# wtedy
# wy
# z
# za
# zaden
# zadna
# zadne
# zadnych
# zapewne
# zawsze
# ze
# zeby
# zeznowu
# zł
# znow
# znowu
# znów
# zostal
# został
# żaden
# żadna
# żadne
# żadnych
# że
# żeby'

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

,abp,abramowic,absurdy,aby,adam,adama,adamowicza,adamowski,adams,adaptację,...,żyć,żyło,żółkiewce,żółkiewki,żółkiewkiks,żółta,żółtoczerwonymi,żółty,żłobka,żłobki
2020-01,0,0,0,0,1,2,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2020-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-03,0,0,0,2,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2020-04,2,0,0,8,3,0,0,0,2,0,...,2,0,0,0,0,0,0,0,0,3
2020-05,1,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2020-06,0,0,0,2,6,0,0,1,0,0,...,0,1,1,1,1,1,0,0,0,0
2020-07,2,0,0,4,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2020-08,0,1,1,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [30]:
data_dtm.to_pickle("dtm.pkl")

In [31]:
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))